In [7]:
# Data Prepare

import pandas as pd
from sklearn.model_selection import ShuffleSplit

from sklearn.utils import shuffle
import re

# df = pd.read_csv('./airline_sentiment_2_w_aa.csv')

DATASET_ENCODING = "ISO-8859-1"
df = pd.read_csv('./training.1600000.processed.noemoticon.csv',encoding=DATASET_ENCODING)
df= df.iloc[:,[0,-1]]
df.columns = ['sentiment','tweet']
df = pd.concat([df.query("sentiment==0").sample(120000),df.query("sentiment==4").sample(120000)])
df.sentiment = df.sentiment.map({0:1,4:0})
df =  shuffle(df).reset_index(drop=True)

# columns = df.columns
# index_X = columns.get_loc('text')
# index_Y = columns.get_loc('airline_sentiment')
# df= df.iloc[:,[index_Y,index_X]]
# df.columns = ['sentiment','tweet']
# df.sentiment = df.sentiment.map({"negative":-1,"positive":1,"neutral":0})
# df =  shuffle(df).reset_index(drop=True)

df.sample(10)


# X = []
# Y = []
# for row in data.values:
#     X.append(row[index_X])
#     Y.append(row[index_Y])

# data = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
# for train, test in data.split(X):
#     x_train, x_test = X[train], X[test]
    # y_train, y_test = Y[train], Y[test]

# print(len(x_test))

# print(X)



,sentiment,tweet
156941,0,I yawn at Palm Pre too. iPhone users unite
115375,0,@texanoutofwater so true!!! @wendywings is the...
14556,1,"@mileycyrus what happened? idk why, but i sen..."
236811,1,"I miss my car, Commander Dickthrust I hope my..."
187629,1,@Gertuzz: Nope.
230495,1,didn't make it through work. Docs app later to...
197082,0,@CabreraNina i havee kwentoo!
215192,1,watching tv via livestream; my tvcard has a pr...
15926,1,"@meltinghalo Awesome! I am work today, double ..."
79415,0,@Cynyma Just dropping by for a twitter hello! ...


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torchtext.vocab import GloVe
import torch
from torchtext import data
from torch.utils.data import DataLoader,Dataset
from nltk.stem import WordNetLemmatizer,PorterStemmer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
import nltk
import pickle
from torch import nn
import torch.nn.functional as F
import string

from sklearn.metrics import roc_auc_score

In [9]:
df.sentiment.value_counts()

1    120000
0    120000
Name: sentiment, dtype: int64

In [10]:
def preprocess_text(df):
    '''Function to preprocess and create corpus'''
    new_corpus=[]
    vocab={}

    for text in tqdm(df["tweet"]):
        tweet = text.maketrans(string.punctuation, ' '*len(string.punctuation))
        words=[w.translate(tweet) for w in re.sub(r'^@[a-zA-Z]+(_[a-zA-Z]+)?','',text).split() ]
        for word in words:
          try:  
            vocab[word]+=1
          except:
            vocab[word]=1

        new_corpus.append(words)
    return new_corpus,vocab

In [11]:
corpus,vocab = preprocess_text(df)

100%|██████████| 240000/240000 [00:03<00:00, 68882.11it/s]


In [12]:
len(vocab)

SEED = 2020
EMBED_SIZE = 200
MAX_FE = 1000
MAX_LEN = 96

In [13]:
def tokenizer(corpus,mode='train'):

  model_save_name = 'tokenizer.pickle'
  path = F"{model_save_name}" 
     

  if mode=="train":
    tokenizer_obj=Tokenizer()
    tokenizer_obj.fit_on_texts(corpus)
    word_index=tokenizer_obj.word_index

    with open(path,'wb') as tok:
      pickle.dump(tokenizer_obj,tok,protocol=pickle.HIGHEST_PROTOCOL)

  else:
 
    word_index=None
    with open('tokenizer.pickle','rb') as tok:
       tokenizer= pickle.load(tok)

  
  sequences=tokenizer_obj.texts_to_sequences(corpus)
  tweet_pad=pad_sequences(sequences,
                            maxlen=MAX_LEN,
                            truncating='post',
                            padding='post')
    
  return tweet_pad,word_index 

In [14]:
tweet_pad,word_index = tokenizer(corpus)

In [15]:
from collections import defaultdict
def prepare_matrix(word_index):
    embedding_dict = GloVe("twitter.27B",dim=200)
    iiv= defaultdict(int)
    oov= defaultdict(int)
    num_words = len(word_index)
    embedding_matrix = np.zeros((num_words+1, 200))

    for word, i in tqdm(word_index.items()):
        if i > num_words:
            continue

        
        emb_vec = embedding_dict[word]
        if not torch.equal(emb_vec,torch.zeros((200),dtype=torch.float)):
          embedding_matrix[i] = emb_vec
          iiv[word]=vocab[word]

        elif torch.equal(embedding_dict[word.lower()],torch.zeros((200),dtype=torch.float)):
          emb_vec = embedding_dict[word.lower()]
          embedding_matrix[i] = emb_vec
          iiv[word]=vocab[word]

        elif torch.equal(embedding_dict[word.title()],torch.zeros((200),dtype=torch.float)):
          emb_vec = embedding_dict[word.title()]
          embedding_matrix[i] = emb_vec
          iiv[word]=vocab[word]

        else:
          oov[word] = vocab[word]

    return embedding_matrix,iiv,oov

In [16]:
embedding_matrix = prepare_matrix(vocab)

100%|██████████| 232196/232196 [00:06<00:00, 38060.04it/s]


In [17]:
X_train,X_test,y_train,y_test = train_test_split(tweet_pad,df.sentiment.values,test_size=0.1)

In [18]:
class TweetModel(nn.Module):

    def __init__(self, embedding_matrix, lstm_hidden_size=200, gru_hidden_size=128):

        super(TweetModel,self).__init__()
        self.embedding = nn.Embedding(*embedding_matrix.shape)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.embedding_dropout = nn.Dropout2d(0.1)

        self.gru = nn.GRU(embedding_matrix.shape[1] , gru_hidden_size,num_layers=1, bidirectional=True, batch_first=True)

        self.dropout2 = nn.Dropout(0.25)
        self.Linear1 = nn.Linear(gru_hidden_size*5,16)
        self.Linear2 = nn.Linear(16,1)


    def forward(self, x):
        batch_size = x.size(0)
        h_embedding = self.embedding(x)

        x, (x_h,x_c) = self.gru(h_embedding)

        avg_pool = torch.mean(x, 1)
        max_pool, _ = torch.max(x, 1)
        concat = torch.cat((avg_pool,x_h,max_pool), 1)
        concat = self.Linear1(concat)
        out = torch.sigmoid(self.Linear2(concat))
        return out

In [19]:
class TweetDataset(Dataset):

  def __init__(self,train,targets=None,mode="train"):

    self.train = train
    self.mode=mode
    self.targets = targets
    
  def __len__(self):

    return len(self.train)

  def __getitem__(self,idx):

    x_train_fold =  torch.tensor(self.train[idx],dtype=torch.long)
    if self.mode=='train':
        y_train_fold = torch.tensor(self.targets[idx],dtype=torch.float32)
        return x_train_fold, y_train_fold
    else:
        return x_train_fold,0

In [15]:
def train_model(train,target,embedding_matrix,nepochs=5,batch_size=64,test_split=0.05):
     
    X_train,X_test,y_train,y_test = train_test_split(train,target,test_size = test_split)

    train_data = TweetDataset(X_train,y_train)
    test_data = TweetDataset(X_test,y_test)

    dataloaders= {"train":DataLoader(train_data,batch_size=batch_size,shuffle=True),
                    "valid":DataLoader(test_data,batch_size=batch_size,shuffle=True)}

    model = TweetModel(embedding_matrix)
    
    loss_fn = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, eps=1e-4, verbose=True)

    best_loss = {'train':np.inf,
                  "valid":np.inf}

    for epoch in range(nepochs):

        epoch_loss = {"train":0.00,
                      "valid":0.00}

        for phase in ['train','valid']:
            if phase=="train":
                model = model.train()
            else:
                model= model.eval()

            running_loss = 0.00

            for i,(x,y) in enumerate(dataloaders['train']):
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase=="train"):

                    predict = model(x).squeeze()
                    loss = loss_fn(predict,y)



                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss+= loss.item()/len(dataloaders[phase])

                epoch_loss[phase]=running_loss

        
        print("Epoch {}/{}   -   loss: {:5.5f}   -   val_loss: {:5.5f}".format(epoch+1, nepochs, epoch_loss['train'], epoch_loss['valid']))
        scheduler.step(epoch_loss['valid'])
        
    return model

In [20]:
model = train_model(X_train,y_train,embedding_matrix[0],nepochs=20)

Epoch 1/20   -   loss: 0.46675   -   val_loss: 6.93234
Epoch 2/20   -   loss: 0.39272   -   val_loss: 5.72838
Epoch 3/20   -   loss: 0.33101   -   val_loss: 4.46998
Epoch 4/20   -   loss: 0.24805   -   val_loss: 2.89526
Epoch 5/20   -   loss: 0.18611   -   val_loss: 2.29871
Epoch 6/20   -   loss: 0.15536   -   val_loss: 1.72280
Epoch 7/20   -   loss: 0.13360   -   val_loss: 1.44530
Epoch 8/20   -   loss: 0.11547   -   val_loss: 1.34587
Epoch 9/20   -   loss: 0.10590   -   val_loss: 1.08955
Epoch 10/20   -   loss: 0.09650   -   val_loss: 1.12091
Epoch 11/20   -   loss: 0.09228   -   val_loss: 0.97987
Epoch 12/20   -   loss: 0.08579   -   val_loss: 0.84572
Epoch 13/20   -   loss: 0.08263   -   val_loss: 0.87146
Epoch 14/20   -   loss: 0.07895   -   val_loss: 0.86188
Epoch 15/20   -   loss: 0.07610   -   val_loss: 0.74716
Epoch 16/20   -   loss: 0.07401   -   val_loss: 0.78346
Epoch 17/20   -   loss: 0.07242   -   val_loss: 0.73497
Epoch 18/20   -   loss: 0.06866   -   val_loss: 0.76498
E

In [23]:
dataloader = DataLoader(TweetDataset(X_test,mode="test"),batch_size=32,shuffle=False)
model.eval()
batch_preds = []
predictions=[]
for x,y in dataloader:
    with torch.no_grad():
        batch_preds = model(x)

    predictions.append(batch_preds)

predictions = torch.cat(predictions, dim=0).cpu().numpy()

In [24]:
print(roc_auc_score(np.round(predictions).squeeze(),y_test))

0.7694197856243459
